In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import time
import scipy.stats
from torch.utils.data import TensorDataset, DataLoader
import optuna
from torch import optim
import copy
import torch.nn.functional as F

Загрузка данных

In [3]:
df = pd.read_csv('daily_accidents.csv', parse_dates=['CRASH DATE'])
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df.set_index('CRASH DATE', inplace=True)
df = df[['ACCIDENT_COUNT']]

Масштабирование данных

In [4]:
df_daily = df.resample('D').sum()

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_daily)

Создание датасета

In [ ]:
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data)-time_step):
        X.append(data[i:(i+time_step), 0])
        y.append(data[i+time_step, 0])
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

X = X.reshape(X.shape[0], X.shape[1], 1)

Создание  обучающей и тестовой выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

if y_train.ndim == 1:
    y_train = y_train.unsqueeze(-1)
    y_test = y_test.unsqueeze(-1)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Определение моделей

In [7]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, num_layers, dropout, bidirectional):
        super(LSTMModel, self).__init__()
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(
            input_size,
            hidden_layer_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        self.fc = nn.Linear(hidden_layer_size * (2 if bidirectional else 1), output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, num_layers, dropout, bidirectional):
        super(GRUModel, self).__init__()
        self.bidirectional = bidirectional
        self.gru = nn.GRU(
            input_size,
            hidden_layer_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        self.fc = nn.Linear(hidden_layer_size * (2 if bidirectional else 1), output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out


Функция подбора гиперпараметров LSTM

In [ ]:
def objective_lstm(trial):
    hidden_size = trial.suggest_int("hidden_layer_size", 32, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])

    model = LSTMModel(
        input_size=1,
        hidden_layer_size=hidden_size,
        output_size=1,
        num_layers=num_layers,
        dropout=dropout,
        bidirectional=bidirectional
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(20):
        for inputs, labels in train_loader:
            inputs, labels = inputs, labels
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

    return test_loss / len(test_loader)


Подбор гиперпараметров LSTM

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_lstm, n_trials=30)

print("Best parameters:", study.best_params)

[I 2025-05-04 00:20:41,683] A new study created in memory with name: no-name-b8de41be-1fe1-44ee-b3e9-5e394b538c9f
[I 2025-05-04 00:21:52,934] Trial 0 finished with value: 0.0008980438137298514 and parameters: {'hidden_layer_size': 140, 'num_layers': 2, 'lr': 0.0002489375809645176, 'dropout': 0.38453349789362296, 'bidirectional': False}. Best is trial 0 with value: 0.0008980438137298514.
[I 2025-05-04 00:23:04,688] Trial 1 finished with value: 0.0008847672845525988 and parameters: {'hidden_layer_size': 101, 'num_layers': 3, 'lr': 0.0005824397571573693, 'dropout': 0.24835239031602713, 'bidirectional': False}. Best is trial 1 with value: 0.0008847672845525988.
[I 2025-05-04 00:28:46,702] Trial 2 finished with value: 0.0009376699955941275 and parameters: {'hidden_layer_size': 175, 'num_layers': 3, 'lr': 0.0004260742382606353, 'dropout': 0.41756737237946734, 'bidirectional': True}. Best is trial 1 with value: 0.0008847672845525988.
[I 2025-05-04 00:30:55,113] Trial 3 finished with value: 0.

Best parameters: {'hidden_layer_size': 241, 'num_layers': 1, 'lr': 0.00010063512681846293, 'dropout': 0.05762019774067895, 'bidirectional': False}


Функция подбора гиперпараметров GRU

In [ ]:
def objective_gru(trial):
    hidden_size = trial.suggest_int("hidden_layer_size", 32, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])

    model = GRUModel(
        input_size=1,
        hidden_layer_size=hidden_size,
        output_size=1,
        num_layers=num_layers,
        dropout=dropout,
        bidirectional=bidirectional
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(20):
        for inputs, labels in train_loader:
            inputs, labels = inputs, labels
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

    return test_loss / len(test_loader)


Подбор гиперпараметров GRU

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_gru, n_trials=30)

print("Best parameters:", study.best_params)

[I 2025-05-04 14:46:59,311] A new study created in memory with name: no-name-b9f3d09e-5ab4-4c5b-b84d-6a7b4983dadd
[I 2025-05-04 14:51:26,514] Trial 0 finished with value: 0.0008021432631408218 and parameters: {'hidden_layer_size': 244, 'num_layers': 1, 'lr': 0.002928224156406466, 'dropout': 0.046592022781241305, 'bidirectional': False}. Best is trial 0 with value: 0.0008021432631408218.
[I 2025-05-04 15:03:30,083] Trial 1 finished with value: 0.001145184663292984 and parameters: {'hidden_layer_size': 160, 'num_layers': 2, 'lr': 0.003893582343785797, 'dropout': 0.3131404994957654, 'bidirectional': True}. Best is trial 0 with value: 0.0008021432631408218.
[I 2025-05-04 15:16:59,355] Trial 2 finished with value: 0.0008221267048141051 and parameters: {'hidden_layer_size': 223, 'num_layers': 3, 'lr': 0.0004609571556194667, 'dropout': 0.21162561926801815, 'bidirectional': False}. Best is trial 0 with value: 0.0008021432631408218.
[I 2025-05-04 15:31:21,424] Trial 3 finished with value: 0.000

Best parameters: {'hidden_layer_size': 170, 'num_layers': 1, 'lr': 0.00012702595787768655, 'dropout': 0.1756996946597152, 'bidirectional': True}


Инициализация моделей

In [ ]:
input_size = 1
hidden_layer_size = 50
output_size = 1

lstm_model = LSTMModel(input_size, output_size=output_size, hidden_layer_size=241, num_layers=1, dropout=0.05762019774067895, bidirectional=False)
gru_model = GRUModel(input_size, output_size=output_size, hidden_layer_size=170, num_layers=1, dropout=0.1756996946597152, bidirectional=True)

criterion = nn.MSELoss()
lstm_optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.00010063512681846293)
gru_optimizer = torch.optim.Adam(gru_model.parameters(), lr=0.00012702595787768655)

Функция обучения

In [9]:
def train_model(model, optimizer, X_train, y_train, epochs=600):
    model.train()
    for epoch in range(epochs):
        output = model(X_train)
        loss = criterion(output, y_train.view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Обучение LSTM

In [10]:
start_time = time.time()
train_model(lstm_model, lstm_optimizer, X_train, y_train)
lstm_time = time.time() - start_time

Обучение GRU

In [11]:
start_time = time.time()
train_model(gru_model, gru_optimizer, X_train, y_train)
gru_time = time.time() - start_time

Прогнозирование и возвращение оригинального масштаба

In [14]:
def predict(model, X_test):
    model.eval()
    with torch.no_grad():
        predicted = model(X_test)
    return predicted

lstm_predicted = predict(lstm_model, X_test)
gru_predicted = predict(gru_model, X_test)

lstm_predicted = scaler.inverse_transform(lstm_predicted.numpy())
gru_predicted = scaler.inverse_transform(gru_predicted.numpy())
y_test_rescaled = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))

Функция вычисление корреляции

In [ ]:
def check_nan(y_true, y_pred):
    if np.any(np.isnan(y_true)) or np.any(np.isnan(y_pred)):
        print("Есть NaN в данных!")
        return True
    return False

def calculate_correlation(y_true, y_pred):
    if check_nan(y_true, y_pred):
        return np.nan 
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    corr, _ = scipy.stats.pearsonr(y_true, y_pred)
    return corr

Оценка по метрикам

In [16]:
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    correlation = calculate_correlation(y_true, y_pred)
    return rmse, mae, r2, correlation

lstm_rmse, lstm_mae, lstm_r2, lstm_corr = evaluate_model(y_test_rescaled, lstm_predicted)
gru_rmse, gru_mae, gru_r2, gru_corr = evaluate_model(y_test_rescaled, gru_predicted)

print(f"LSTM RMSE: {lstm_rmse}, MAE: {lstm_mae}, R²: {lstm_r2}, Correlation: {lstm_corr}")
print(f"GRU RMSE: {gru_rmse}, MAE: {gru_mae}, R²: {gru_r2}, Correlation: {gru_corr}")

LSTM RMSE: 31.08243748359956, MAE: 24.109880447387695, R²: 0.22560328245162964, Correlation: 0.492148756980896
GRU RMSE: 31.824012337077622, MAE: 24.896503448486328, R²: 0.1882108449935913, Correlation: 0.5239609479904175


Время на обучение моделей

In [19]:
print(f"Время обучения LSTM: {lstm_time} секунд")

Время обучения LSTM: 7763.463284492493 секунд


In [20]:
print(f"Время обучения GRU: {gru_time} секунд")

Время обучения GRU: 5991.344755887985 секунд
